In [1]:
import numpy as np  # linear algebra
import pandas as pd  # CSV file
import scipy.io.wavfile as sci_wav  # Open wav files
import matplotlib.pyplot as plt
import sounddevice as sd
from scipy.io import wavfile
from scipy.fftpack import dct
import librosa
import os
import config


* Load The WAV files!!!!!!!!!

In [2]:
#Change it to your root directory where you downloaded the dataset
#protip: (make a config.py file and add the path in your machine to it. Name the variable as ROOT_DIR )

ROOT_DIR = config.ROOT_DIR 

# List all files and folders in the directory
all_entries = os.listdir(ROOT_DIR)

# If you want to list only files (excluding directories)
files = [f for f in all_entries if os.path.isfile(os.path.join(ROOT_DIR, f))]

# Print the result
print("All entries:", all_entries)
print("Files only:", files)

cats = [f for f in files if f.startswith('cat')]
dogs = [f for f in files if f.startswith('dog')]

print(f"cats path: {cats}")
print(f"dogs path: {dogs}")

All entries: ['cat_1.wav', 'cat_10.wav', 'cat_100.wav', 'cat_101.wav', 'cat_102.wav', 'cat_103.wav', 'cat_105.wav', 'cat_106.wav', 'cat_107.wav', 'cat_108.wav', 'cat_109.wav', 'cat_11.wav', 'cat_110.wav', 'cat_112.wav', 'cat_113.wav', 'cat_114.wav', 'cat_115.wav', 'cat_116.wav', 'cat_117.wav', 'cat_118.wav', 'cat_119.wav', 'cat_12.wav', 'cat_120.wav', 'cat_121.wav', 'cat_122.wav', 'cat_123.wav', 'cat_124.wav', 'cat_125.wav', 'cat_126.wav', 'cat_127.wav', 'cat_128.wav', 'cat_129.wav', 'cat_13.wav', 'cat_130.wav', 'cat_131.wav', 'cat_132.wav', 'cat_133.wav', 'cat_134.wav', 'cat_135.wav', 'cat_136.wav', 'cat_137.wav', 'cat_138.wav', 'cat_139.wav', 'cat_14.wav', 'cat_140.wav', 'cat_141.wav', 'cat_142.wav', 'cat_143.wav', 'cat_144.wav', 'cat_146.wav', 'cat_147.wav', 'cat_148.wav', 'cat_149.wav', 'cat_15.wav', 'cat_150.wav', 'cat_151.wav', 'cat_152.wav', 'cat_153.wav', 'cat_154.wav', 'cat_155.wav', 'cat_156.wav', 'cat_157.wav', 'cat_158.wav', 'cat_159.wav', 'cat_16.wav', 'cat_160.wav', 'cat_

In [3]:
print(f"{len(cats)=} files")
print(f"{len(dogs)=} files")

len(cats)=164 files
len(dogs)=113 files


* Check the total length of samples.

In [4]:
def Length_of_wavs(wavs: list, root: str ) -> tuple:
    """
    We wanna check if the durations pf cats and dogs audio
    are comparable to avoid umbalanced dataset!! 
    
    Additionally, lets also chack the sample rate of 
    each file to make sure there are no outliers.
    
    """
    duration = 0
    rates = []
    for i in wavs:
        rate, data = wavfile.read(root + i)
        rates.append(rate)
        duration += len(data) / float(rate)
    return(duration, set(rates))              #The set() function is used to check if the values in the list are unique

duration_cats, Sampleratescats = Length_of_wavs(cats,ROOT_DIR)
duration_dogs, Sampleratesdogs = Length_of_wavs(dogs,ROOT_DIR)
print(f"Duration of cats : {duration_cats:.2f} seconds")
print(f"Duration of dogs : {duration_dogs:.2f} seconds")
print(f"Unique sample rates = {Sampleratescats, Sampleratesdogs}")



Duration of cats : 1323.90 seconds
Duration of dogs : 598.44 seconds
Unique sample rates = ({16000}, {16000})


* Lets try to get speech only parts from the audios (Have to figure this out)

In [5]:
# from vad import EnergyVAD              #I think this apprach is based on energy density in the signal to detect the speech intervals
# import soundfile as sf

# test_audio = ROOT_DIR + 'cat_25.wav'    #A random exampple
# saveas = config.SavePath  #path wwhere you wanna save the processed file

# #play the example
# Srate, audio = wavfile.read(test_audio)
# sd.play(audio, Srate)
# sd.wait()




In [6]:
# #Try to eliminate long pauses

# audio, sr = sf.read(test_audio)
# if audio.ndim == 1:
#     audio = np.expand_dims(audio, axis=0)  # Shape becomes (1, num_samples)
# vad = EnergyVAD(sample_rate=sr)
# speech_signal = vad.apply_vad(audio)

# #Try to add pause

# # Define silence duration in seconds
# padding_duration = 1.0 
# padding_samples = int(sr * padding_duration)

# # Create silence
# silence = np.zeros(padding_samples)

# print("Speech signal shape:", speech_signal.shape)
# speech_signal = speech_signal.flatten()
# speech_signal = speech_signal.astype('float32')
# # Add silence to beginning and end
# padded_signal = np.concatenate([silence, speech_signal, silence])

# sf.write(saveas, padded_signal, sr)

In [7]:
# #TO DO (Looks like the pauses at the begining and end are eliminated.Would be nice to include that as well )

# Srate, audio = wavfile.read(saveas)
# print(Srate)
# sd.play(audio, Srate)
# sd.wait()


* ################################# Get the mel Coeficients ##################################################

In [8]:
def compute_mfcc(file):

    y, sr = librosa.load(file)
    #######################################
    pre_emphasis = 0.97
    y_preemphasized = np.append(y[0], y[1:] - pre_emphasis * y[:-1])
    #######################################
    frame_size = 0.025  # 25 ms
    frame_stride = 0.01  # 10 ms
    frame_length, frame_step = frame_size * sr, frame_stride * sr  # Convert from seconds to samples
    signal_length = len(y_preemphasized)
    frame_length = int(round(frame_length))
    frame_step = int(round(frame_step))
    num_frames = int(np.ceil(float(np.abs(signal_length - frame_length)) / frame_step))
    ###########################################################################
    # Pad signal to ensure all frames have equal number of samples
    pad_signal_length = num_frames * frame_step + frame_length
    z = np.zeros((pad_signal_length - signal_length))
    pad_signal = np.append(y_preemphasized, z)
    ###############################################################
    # Slice the signal into frames
    indices = np.tile(np.arange(0, frame_length), (num_frames, 1)) + np.tile(np.arange(0, num_frames * frame_step, frame_step), (frame_length, 1)).T
    frames = pad_signal[indices.astype(np.int32, copy=False)]
    #############################################################################
    frames *= np.hamming(frame_length)
    ################################################################################
    NFFT = 512
    mag_frames = np.absolute(np.fft.rfft(frames, NFFT))  # Magnitude of the FFT
    pow_frames = ((1.0 / NFFT) * ((mag_frames) ** 2))  # Power Spectrum
    #################################################################################
    nfilt = 40
    low_freq_mel = 0
    high_freq_mel = 2595 * np.log10(1 + (sr / 2) / 700)  # Convert Hz to Mel
    mel_points = np.linspace(low_freq_mel, high_freq_mel, nfilt + 2)  # Equally spaced in Mel scale
    hz_points = 700 * (10 ** (mel_points / 2595) - 1)  # Convert Mel to Hz
    bin = np.floor((NFFT + 1) * hz_points / sr)
    fbank = np.zeros((nfilt, int(np.floor(NFFT / 2 + 1))))
    for m in range(1, nfilt + 1):
        f_m_minus = int(bin[m - 1])   # left
        f_m = int(bin[m])             # center
        f_m_plus = int(bin[m + 1])    # right

        for k in range(f_m_minus, f_m):
            fbank[m - 1, k] = (k - bin[m - 1]) / (bin[m] - bin[m - 1])
        for k in range(f_m, f_m_plus):
            fbank[m - 1, k] = (bin[m + 1] - k) / (bin[m + 1] - bin[m])

    filter_banks = np.dot(pow_frames, fbank.T)
    filter_banks = np.where(filter_banks == 0, np.finfo(float).eps, filter_banks)  # Numerical stability
    filter_banks = 20 * np.log10(filter_banks)  # dB
    ######################################################################################
    num_ceps = 12
    mfcc = dct(filter_banks, type=2, axis=1, norm='ortho')[:, :num_ceps]

    return mfcc

In [9]:
test_audio = ROOT_DIR + 'cat_25.wav' 
Test_Mfcc = compute_mfcc(test_audio)
print(len(Test_Mfcc))

1171


* ========== Compute MFCC for all audio files and save the 12 CC features with labels =======================

In [10]:
def Label_and_Save(Mfcc, label):

    Root = config.Features
    label_cat = 0  
    label_dog = 1
    num_frames = Mfcc.shape[0]

    if label == 'cat':
        labels = np.full((num_frames, 1), label_cat)
        features_with_labels = np.hstack((Mfcc, labels))
    
    elif label == "dog":
        labels = np.full((num_frames, 1), label_dog)
        features_with_labels = np.hstack((Mfcc, labels))


    # csv_test = config.Features
    df = pd.DataFrame(features_with_labels)
    #df.to_csv(Root + "mfcc_features.csv", index=False, header=False)
    df.to_csv(Root + 'data.csv', mode='a', header=False, index=False)



In [ ]:
#Compute MFCC and append to csv

label_dog = 'dog'
label_cat = 'cat'

for dog in dogs:
    audio = ROOT_DIR + dog 
    MFCC = compute_mfcc(audio)
    Label_and_Save(Mfcc= MFCC, label=label_dog)

for cat in cats:
    audio = ROOT_DIR + cat 
    MFCC = compute_mfcc(audio)
    Label_and_Save(Mfcc= MFCC, label=label_cat)

    
    
    